In [1]:
pip install validators

In [2]:
pip install tldextract

     |████████████████████████████████| 87 kB 3.0 MB/s 


In [3]:
import pandas as pd
import numpy as np
from xml.etree import cElementTree as ET
from lxml import html
import random
import pandas as pd
import validators
import requests
from tldextract import extract

In [4]:
tree = ET.parse('/content/drive/MyDrive/Posts.xml')

In [5]:
root = tree.getroot()

In [6]:
rootlen = len(root)

In [7]:
listOfUrl = []
for i in range(rootlen):
    #print("IdContent:",root[i].attrib['Id'],"\tBodyContent:",root[i].attrib['Body'])
    if root[i].attrib['Body']:
        ht= html.fromstring(root[i].attrib['Body'])
     
        htlist=ht.xpath('//a/@href') # if the htlist is empty, it would be considered as 'False' in next 'if' step
        if htlist:
            #print(i)
            #listOfUrl.append(ht.xpath('//a/@href'))
            listOfUrl.extend(ht.xpath('//a/@href'))
        else:
            pass

In [8]:
listOfUrl # Dataset A

['https://data.stackexchange.com/stackoverflow/query/157285/meta-reputation-classifications',
 'http://chat.meta.stackoverflow.com/rooms/89/tavern-on-the-meta',
 'https://chat.stackexchange.com/?tab=site&host=meta.stackoverflow.com',
 'https://i.imgur.com/j9ZpZs4.png',
 '/questions/tagged/featured',
 'https://stackoverflow.com/help/whats-meta',
 'https://meta.stackoverflow.com/questions/250976/migrations-are-now-in-progress',
 'https://blog.stackoverflow.com/2014/04/announcing-the-launch-of-meta-stack-exchange/',
 'https://stackoverflow.com/help/badges/901/convention',
 'https://stackoverflow.com/election/5',
 'https://stackoverflow.com/help/badges/901/convention',
 'https://meta.stackoverflow.com/',
 'https://meta.stackoverflow.com/',
 'http://meta.stackexchange.com/q/228888/',
 'https://meta.stackoverflow.com/questions/250068/february-2014-stack-overflow-community-moderator-election-results',
 'https://stackexchange.com/sites',
 'https://stackoverflow.com/help',
 'http://meta.stackex

In [9]:
def choose_randomly (N):
    ListofNurl = []
    for i in range(N):
        ListofNurl.append(random.choice(listOfUrl))
    
    return ListofNurl

In [10]:
#filtering list of the non-clickable links
def remove_non_clickable(filtered_listof_N_url):
    remove_listof_N_url = []
    for i in range(len(filtered_listof_N_url)):
      if not validators.url(filtered_listof_N_url[i]):
          remove_listof_N_url.append(filtered_listof_N_url[i])
          #filtered_listof_N_url.remove(filtered_listof_N_url[i])

    #removing invalid address
    j=0
    for j in range(len(remove_listof_N_url)):
        filtered_listof_N_url.remove(remove_listof_N_url[j])

    return filtered_listof_N_url # list of clickable links only

In [11]:
#identifying not working address
def remove_non_working(filtered_listof_N_url):
      notworking_listof_N_url = []
      notworking_listof_N_url2 = []
      for i in range(len(filtered_listof_N_url)):
          try:
              response = requests.get(filtered_listof_N_url[i])
              if response.status_code == 404:  #page 404 error
                  notworking_listof_N_url.append(filtered_listof_N_url[i])
                  #filtered_listof_N_url.remove(filtered_listof_N_url[i])
          except requests.ConnectionError as exception:
                  notworking_listof_N_url2.append(filtered_listof_N_url[i]) #This site can’t be reached error
          

      #removing 404 not working address
      j=0
      for j in range(len(notworking_listof_N_url)):
          filtered_listof_N_url.remove(notworking_listof_N_url[j])

      #removing exception working address
      j=0
      for j in range(len(notworking_listof_N_url2)):
          filtered_listof_N_url.remove(notworking_listof_N_url2[j])

      return filtered_listof_N_url  # list of working links only

In [12]:
#counting repetitions/citations
def count_repetitions(filtered_listof_N_url):
    count_listof_N_url = filtered_listof_N_url.copy()
    i=0
    for i in range(len(filtered_listof_N_url)):
        count_listof_N_url[i]= listOfUrl.count(filtered_listof_N_url[i]) # computing count of each URL in listOfUrl (Dataset A)

    return count_listof_N_url

In [ ]:
#RQ3

In [13]:
#Datset B
Clickable_listof_N_url = remove_non_clickable(listOfUrl) #getting clickable links from Dataset A 

In [14]:
count_listof_N_url = count_repetitions(Clickable_listof_N_url)

In [15]:
len(Clickable_listof_N_url)

171503

In [16]:
len(count_listof_N_url)

171503

In [15]:
#combining URL list and Repetition of URL List into a dict. This simultaneously drops any duplicates
#this code will be reused in RQ2
result_dict = {Clickable_listof_N_url[i]: count_listof_N_url[i] for i in range(len(Clickable_listof_N_url))}

#converting above dict into dataframe
df = pd.DataFrame([[k, v] for k, v in result_dict.items()], columns=['URL Name', 'Repetition'])
df.to_csv('/content/drive/MyDrive/ALL_valid_url_with_repetition.csv')

In [18]:
#extracting domains from URL Name of above df
domain_list = [None] * len(df)
sub_domain_list = [None] * len(df)
i=0
for i in range(len(df)):
    tsd, td, tsu = extract(df['URL Name'][i]) # prints subhostname, hostname, com
    domain_list[i] = td + '.' + tsu # will prints as hostname.com
    sub_domain_list[i] = tsd # prints subhostname

In [19]:
df.insert(1,"URL Sub Domain",sub_domain_list,True)

In [20]:
df.insert(2,"URL Domain",domain_list,True)

In [21]:
df.head()

,URL Name,URL Sub Domain,URL Domain,Repetition
0,https://data.stackexchange.com/stackoverflow/q...,data,stackexchange.com,1
1,http://chat.meta.stackoverflow.com/rooms/89/ta...,chat.meta,stackoverflow.com,2
2,https://chat.stackexchange.com/?tab=site&host=...,chat,stackexchange.com,2
3,https://i.imgur.com/j9ZpZs4.png,i,imgur.com,1
4,https://stackoverflow.com/help/whats-meta,,stackoverflow.com,67


In [22]:
#counting the URLs in each domain

count_domain_list = [None] * len(df)
i=0
for i in range(len(df)):
    count_domain_list[i]= domain_list.count(domain_list[i])

In [23]:
df.insert(3,"Unique URL Count in Domain ",count_domain_list,True)

In [24]:
df.head()

,URL Name,URL Sub Domain,URL Domain,Unique URL Count in Domain,Repetition
0,https://data.stackexchange.com/stackoverflow/q...,data,stackexchange.com,10545,1
1,http://chat.meta.stackoverflow.com/rooms/89/ta...,chat.meta,stackoverflow.com,73275,2
2,https://chat.stackexchange.com/?tab=site&host=...,chat,stackexchange.com,10545,2
3,https://i.imgur.com/j9ZpZs4.png,i,imgur.com,16102,1
4,https://stackoverflow.com/help/whats-meta,,stackoverflow.com,73275,67


In [26]:
df.drop('URL Sub Domain', axis=1, inplace=True)

In [27]:
df.drop('URL Name', axis=1, inplace=True)

In [28]:
df.head()

,URL Domain,Unique URL Count in Domain,Repetition
0,stackexchange.com,10545,1
1,stackoverflow.com,73275,2
2,stackexchange.com,10545,2
3,imgur.com,16102,1
4,stackoverflow.com,73275,67


In [29]:
# Sum of repititons under each domain

i=0
j=0
sum_domain_repetition = [None] * len(df)
track_domain_name = []

for i in range(len(df)):
    domain_name = df['URL Domain'][i]
    repetition_count = 0
    if domain_name not in track_domain_name:
        track_domain_name.append(domain_name)
        for j in range(len(df)):
            if df['URL Domain'][j] == domain_name:
                repetition_count += df['Repetition'][j]
        sum_domain_repetition[i] = repetition_count

In [30]:
len(df)

114185

In [31]:
len(sum_domain_repetition)

114185

In [32]:
df.insert(2,"Total Citations in Domain",sum_domain_repetition,True)

In [33]:
df.drop_duplicates(subset='URL Domain', keep='first', inplace=True) ##removes records that have same column value , for column_name in (subset='URL Domain'), preserving only the (keep='first')record

In [34]:
df.rename(columns={'Unique URL Count in Domain ': 'Unique URL Count Domain'}, inplace=True)# extra space in the colum name is removed

In [35]:
len(df)

2828

In [66]:
#df2 = df.copy()

In [71]:
df=df.reset_index()

In [73]:
df.drop('index', axis=1, inplace=True)

In [74]:
df.drop('Repetition', axis=1, inplace=True)

In [76]:
#calculating the ratio
percent_list = [None] * len(df)
i=0
for i in range(len(df)):
      percent_list[i] = df['Unique URL Count Domain'][i] / df['Total Citations in Domain'][i]

In [77]:
df.insert(3,"%",percent_list,True)

In [78]:
df = df.sort_values(by='Total Citations in Domain', ascending=False)

In [ ]:
df.to_csv('/content/drive/MyDrive/ALL_valid_url_RQ3_Complete.csv')

In [ ]:
#Dataset for training the ML model 

In [80]:
List_500_random_links = choose_randomly(500)

In [81]:
Clickable_listof_500_url = remove_non_clickable(List_500_random_links)

In [82]:
Working_clickable_listof_500_url = remove_non_working(Clickable_listof_500_url)

In [83]:
len(Working_clickable_listof_500_url)

467

In [ ]:
dict = {"URL Name": Working_clickable_listof_500_url}
df_500 = pd.DataFrame(dict)
df_500.to_csv('/content/drive/MyDrive/Working_clickable_listof_500_url.csv') #this data will be manually labelled and used for training the model

In [ ]:
#the model traning code is separate file

In [ ]:
#RQ1 WebisteType

In [89]:
List_5000_random_links = choose_randomly(5000)

In [91]:
Clickable_listof_5000_url = remove_non_clickable(List_5000_random_links)

In [92]:
Working_clickable_listof_5000_url = remove_non_working(Clickable_listof_5000_url)

In [ ]:
dict = {"URL Name": Working_clickable_listof_5000_url}
df_5000 = pd.DataFrame(dict)
df_5000.to_csv('/content/drive/MyDrive/testset_2_UnknownLabels.csv')#WebsiteType and WebsiteContentType will be predicted for this dataset

In [ ]:
#WebsiteType will be predicted at a separate collab file: Prediction_Model_Web_Types_Project_SE
#After prediction, we will receive a file named : PredictedWebTypes

In [93]:
df_Predicted_WebType = pd.read_csv('/content/drive/MyDrive/PredictedWebTypes.csv',sep = ',')

In [94]:
df_Predicted_WebType = df_Predicted_WebType.head(4000)

In [96]:
df_Predicted_WebType = df_Predicted_WebType.groupby(['decoded_web_type']).size().reset_index(name='count')

In [97]:
df_Predicted_WebType = df_Predicted_WebType.sort_values(by='count', ascending=False)

In [98]:
df_Predicted_WebType.head(10)

,decoded_web_type,count
5,Q&A Post,2933
1,Images,468
3,Other,252
6,User Profile,119
0,Blog Post,109
2,Official Documentation,46
7,Wiki,45
4,Product/Project Website,28


In [106]:
df_Predicted_WebType.to_csv('/content/drive/MyDrive/random_4k_RQ1_WebType_Complete.csv')

In [ ]:
#WebsiteContentType will be predicted in a separate collab file: Prediction_Model_Web_Content_Project_SE
#After prediction, we will receive a file named : PredictedWebContenton4000Unknown

In [5]:
df_Predicted_WebContentType = pd.read_csv('/content/drive/MyDrive/PredictedWebContenton4000Unknown.csv',sep = ',')

In [6]:
df_Predicted_WebContentType = df_Predicted_WebContentType.head(4000)

In [7]:
df_Predicted_WebContentType.head()

,Unnamed: 0,Unnamed: 0.1,URL Name,Predicted Web Content,Decoded Web Content
0,0,0,https://data.stackexchange.com/stackoverflow/q...,3,Question Answers
1,1,1,https://stackoverflow.com/questions/tagged/mic...,3,Question Answers
2,2,3,http://appscript.sourceforge.net/,2,Other
3,3,4,https://meta.stackoverflow.com/questions/25265...,3,Question Answers
4,4,5,https://meta.stackoverflow.com/questions/33380...,3,Question Answers


In [8]:
df_Predicted_WebContentType = df_Predicted_WebContentType.groupby(['Decoded Web Content']).size().reset_index(name='count')

In [9]:
df_Predicted_WebContentType = df_Predicted_WebContentType.sort_values(by='count', ascending=False)

In [10]:
df_Predicted_WebContentType.head(10)

,Decoded Web Content,count
3,Question Answers,2293
2,Other,716
0,Graphical Content,468
1,Opinion/Specialty,238
4,Reference,133
6,User Outline,132
5,Tools/Libraries,20


In [12]:
df_Predicted_WebContentType.to_csv('/content/drive/MyDrive/random_4k_RQ1_WebContentType_Complete.csv')

In [ ]:
#RQ2: Website Type

In [16]:
result_dict = {Clickable_listof_N_url[i]: count_listof_N_url[i] for i in range(len(Clickable_listof_N_url))}

#converting above dict into dataframe
df_RQ2 = pd.DataFrame([[k, v] for k, v in result_dict.items()], columns=['URL Name', 'Repetition'])

In [17]:
df_RQ2.head()

,URL Name,Repetition
0,https://data.stackexchange.com/stackoverflow/q...,1
1,http://chat.meta.stackoverflow.com/rooms/89/ta...,2
2,https://chat.stackexchange.com/?tab=site&host=...,2
3,https://i.imgur.com/j9ZpZs4.png,1
4,https://stackoverflow.com/help/whats-meta,67


In [18]:
df_RQ2 = df_RQ2.sort_values(by='Repetition', ascending=False)

In [19]:
df_RQ2.head()

,URL Name,Repetition
924,https://stackoverflow.com/help/on-topic,574
122,https://stackoverflow.com/help/mcve,557
1358,https://stackoverflow.com/help/how-to-ask,461
1287,https://stackoverflow.com/questions/tagged/java,461
1291,https://stackoverflow.com/questions/tagged/jav...,435


In [20]:
df_300 = df_RQ2.head(300) # top 300 clickable links

In [21]:
filtered_listof300url = []
filtered_listof300url = list(df_300['URL Name'])

In [22]:
Working_clickable_listof_300_url = remove_non_working(filtered_listof300url)

In [118]:
# Out of the 300 links, consider only the Working links as in list : Working_clickable_listof_300_url
df_filtered = df_300[df_300['URL Name'].isin(Working_clickable_listof_300_url)]

In [119]:
df_top_100 = df_filtered.head(100) # top 100 most frequently cited URLs

In [ ]:
df_top_100.to_csv('/content/drive/MyDrive/top100_url.csv')

In [ ]:
#WebsiteType for links in top100_url.csv will be predicted at a diferrent collab file: Prediction_Model_Web_Types_Project_SE
#After prediction, we will receive a file named : PredictedWebTypesForTop100Urls

In [23]:
df_top100_WebType = pd.read_csv('/content/drive/MyDrive/PredictedWebTypesForTop100Urls.csv', sep = ',')

In [26]:
df_top100_WebType.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','URL Sub Domain'],axis=1, inplace=True)

In [27]:
df_top100_WebType

,URL Name,URL Domain,Repetition,Encoded Web types,Decoded Web Type
0,https://stackoverflow.com/help/on-topic,stackoverflow.com,574,0,Blog Post
1,https://stackoverflow.com/help/mcve,stackoverflow.com,557,0,Blog Post
2,https://stackoverflow.com/help/how-to-ask,stackoverflow.com,461,0,Blog Post
3,https://stackoverflow.com/questions/tagged/java,stackoverflow.com,461,5,Q&A Post
4,https://stackoverflow.com/questions/tagged/jav...,stackoverflow.com,435,5,Q&A Post
...,...,...,...,...,...
95,https://stackoverflow.com/questions/tagged/string,stackoverflow.com,48,5,Q&A Post
96,https://stackoverflow.com/questions/tagged/css3,stackoverflow.com,47,5,Q&A Post
97,http://creativecommons.org/licenses/by-sa/3.0/,creativecommons.org,47,3,Other
98,https://stackoverflow.com/help/privileges/crea...,stackoverflow.com,47,0,Blog Post


In [28]:
list2 = []
list2 = df_top100_WebType.groupby(['Decoded Web Type']).size()

In [30]:
df_top100_WebType.drop(['URL Name','URL Domain','Encoded Web types'],axis=1, inplace=True)

In [31]:
i=0
j=0
webtype_count = [None] * len(df_top100_WebType)
webtype_citataions = [None] * len(df_top100_WebType)
webtype_name = []

for i in range(len(df_top100_WebType)):
    domain_name = df_top100_WebType['Decoded Web Type'][i]
    webtype_count1 = 0
    webtype_citataions1 = 0
    if domain_name not in webtype_name:
        webtype_name.append(domain_name)
        for j in range(len(df_top100_WebType)):
            if df_top100_WebType['Decoded Web Type'][j] == domain_name:
                webtype_citataions1 += df_top100_WebType['Repetition'][j]
                webtype_count1 += 1
        webtype_citataions[i] = webtype_citataions1
        webtype_count[i] = webtype_count1

In [34]:
df_top100_WebType.insert(2,"WebType Count",webtype_count,True)

In [35]:
df_top100_WebType.insert(3,"WebType Citations",webtype_citataions,True)

In [38]:
df_top100_WebType.drop('Repetition', axis=1, inplace=True)

In [40]:
df_top100_WebType.rename(columns={'Decoded Web Type': 'Web Type'}, inplace=True)

In [42]:
df_top100_WebType.drop_duplicates(subset='Web Type', keep='first', inplace=True)

In [44]:
df_top100_WebType = df_top100_WebType.sort_values(by='WebType Citations', ascending=False)

In [49]:
df_top100_WebType = df_top100_WebType.reset_index()

In [51]:
df_top100_WebType.drop(['index'],axis=1, inplace=True)

In [46]:
df_top100_WebType.loc['4'] = ['Official Documentation', '0', '0']
df_top100_WebType.loc['5'] = ['Images', '0', '0']
df_top100_WebType.loc['6'] = ['Product/Project Website', '0', '0']
df_top100_WebType.loc['7'] = ['Wiki', '0', '0']

In [52]:
df_top100_WebType.head(10)

,Web Type,WebType Count,WebType Citations
0,Q&A Post,63,7374
1,Blog Post,28,4041
2,Other,6,452
3,User Profile,3,178
4,Official Documentation,0,0
5,Images,0,0
6,Product/Project Website,0,0
7,Wiki,0,0


In [66]:
df_top100_WebType.to_csv('/content/drive/MyDrive/top100_RQ2_Complete.csv')

In [ ]:
#WebsiteContentType for links in top100_url.csv will be predicted at a diferrent collab file: Prediction_Model_Web_Content_Project_SE
#After prediction, we will receive a file named : PredictedWebContentTypesForTop100Urls

In [53]:
df_top100_WebConType = pd.read_csv('/content/drive/MyDrive/PredictedWebContentonTop100URLsUnknown.csv', sep = ',')

In [54]:
df_top100_WebConType.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','URL Sub Domain'],axis=1, inplace=True)

In [56]:
df_top100_WebConType.head(5)

,URL Name,URL Domain,Repetition,Predicted Web Content,Decoded Web Content
0,https://stackoverflow.com/help/on-topic,stackoverflow.com,574,1,Opinion/Specialty
1,https://stackoverflow.com/help/mcve,stackoverflow.com,557,1,Opinion/Specialty
2,https://stackoverflow.com/help/how-to-ask,stackoverflow.com,461,1,Opinion/Specialty
3,https://stackoverflow.com/questions/tagged/java,stackoverflow.com,461,3,Question Answers
4,https://stackoverflow.com/questions/tagged/jav...,stackoverflow.com,435,3,Question Answers


In [57]:
list2 = []
list2 = df_top100_WebConType.groupby(['Decoded Web Content']).size()

In [59]:
df_top100_WebConType.drop(['URL Name','URL Domain','Predicted Web Content'],axis=1, inplace=True)

In [61]:
i=0
j=0
webtype_count = [None] * len(df_top100_WebConType)
webtype_citataions = [None] * len(df_top100_WebConType)
webtype_name = []

for i in range(len(df_top100_WebConType)):
    domain_name = df_top100_WebConType['Decoded Web Content'][i]
    webtype_count1 = 0
    webtype_citataions1 = 0
    if domain_name not in webtype_name:
        webtype_name.append(domain_name)
        for j in range(len(df_top100_WebConType)):
            if df_top100_WebConType['Decoded Web Content'][j] == domain_name:
                webtype_citataions1 += df_top100_WebConType['Repetition'][j]
                webtype_count1 += 1
        webtype_citataions[i] = webtype_citataions1
        webtype_count[i] = webtype_count1

In [62]:
df_top100_WebConType.insert(2,"WebContentType Count",webtype_count,True)
df_top100_WebConType.insert(3,"WebContentType Citations",webtype_citataions,True)
df_top100_WebConType.drop('Repetition', axis=1, inplace=True)
df_top100_WebConType.rename(columns={'Decoded Web Content': 'WebContent Type'}, inplace=True)
df_top100_WebConType.drop_duplicates(subset='WebContent Type', keep='first', inplace=True)
df_top100_WebConType = df_top100_WebConType.sort_values(by='WebContentType Citations', ascending=False)
df_top100_WebConType = df_top100_WebConType.reset_index()
df_top100_WebConType.drop(['index'],axis=1, inplace=True)


In [64]:
df_top100_WebConType.loc['5'] = ['Graphical Content', '0', '0']
df_top100_WebConType.loc['6'] = ['Tools/Libraries', '0', '0']

In [65]:
df_top100_WebConType.head(10)

,WebContent Type,WebContentType Count,WebContentType Citations
0,Question Answers,49,6138
1,Opinion/Specialty,29,4145
2,Other,18,1537
3,User Outline,3,178
4,Reference,1,47
5,Graphical Content,0,0
6,Tools/Libraries,0,0


In [67]:
df_top100_WebConType.to_csv('/content/drive/MyDrive/top100_RQ2_WebContentType_Complete.csv')